In [3]:
# # code for loading the format for the notebook
import os

# # path : store the current path to convert back to it later
# path = os.getcwd()
# os.chdir(os.path.join('..', 'notebook_format'))

# from formats import load_style
# load_style(plot_style=False)

In [4]:
# os.chdir(path)

# # 1. magic to print version
# # 2. magic so that the notebook will reload external python modules
# %load_ext watermark
# %load_ext autoreload
# %autoreload 2

import math
import time
import logging
import requests
import threading
import multiprocessing
import concurrent.futures
from joblib import Parallel, delayed

# %watermark -a 'Ethen' -d -t -v -p joblib,requests

In [5]:
def sleeper(n_time):
    name = threading.current_thread().name
    print('I am {}. Going to sleep for {} seconds'.format(name, n_time))
    time.sleep(n_time)
    print('{} has woken up from sleep'.format(name))

In [6]:
# we call .start to start executing the function from the thread
n_time = 2
thread = threading.Thread(target = sleeper, name = 'thread1', args = (n_time,))
thread.start()

I am thread1. Going to sleep for 2 seconds


thread1 has woken up from sleep


In [7]:
# hello is printed "before" the wake up message from the function
thread = threading.Thread(target = sleeper, name = 'thread2', args = (n_time,))
thread.start()

print()
print('hello')

I am thread2. Going to sleep for 2 seconds

hello


thread2 has woken up from sleep


In [8]:
# hello is printed "after" the wake up message from the function
thread = threading.Thread(target = sleeper, name = 'thread3', args = (n_time,))
thread.start()
thread.join()

print()
print('hello')

I am thread3. Going to sleep for 2 seconds
thread3 has woken up from sleep

hello


In [9]:
n_time = 2
n_threads = 5
start = time.time()

# create n_threads number of threads and store them in a list
threads = []
for i in range(n_threads):
    name = 'thread{}'.format(i)
    thread = threading.Thread(target = sleeper, name = name, args = (n_time,))
    threads.append(thread)
    # we can start the thread while we're creating it, or move
    # this to its own loop (as shown below)
    thread.start()

# we could instead start the thread in a separate loop
# for thread in threads:
#     thread.start()

# ensure all threads have finished before executing main program
for thread in threads:
    thread.join()

elapse = time.time() - start
print()
print('Elapse time: ', elapse)

I am thread0. Going to sleep for 2 seconds
I am thread1. Going to sleep for 2 seconds
I am thread2. Going to sleep for 2 seconds
I am thread3. Going to sleep for 2 seconds
I am thread4. Going to sleep for 2 seconds
thread0 has woken up from sleep
thread1 has woken up from sleep
thread3 has woken up from sleep
thread4 has woken up from sleep
thread2 has woken up from sleep

Elapse time:  2.003207206726074


In [10]:
# example from the documentation page
# https://docs.python.org/3/library/concurrent.futures.html#processpoolexecutor-example
def is_prime(n):
    """
    References
    ----------
    https://math.stackexchange.com/questions/1343171/why-only-square-root-approach-to-check-number-is-prime
    """
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False

    return True


PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

with concurrent.futures.ProcessPoolExecutor() as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
        print('{} is prime: {}'.format(number, prime))

112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False


In [11]:
def only_sleep():
    """Wait for a timer to expire"""
    process_name = multiprocessing.current_process().name
    thread_name = threading.current_thread().name
    print('Process Name: {}, Thread Name: {}'.format(
        process_name, thread_name))
    
    time.sleep(4)


def crunch_numbers():
    """Do some computations """
    process_name = multiprocessing.current_process().name
    thread_name = threading.current_thread().name
    print('Process Name: {}, Thread Name: {}'.format(
        process_name, thread_name))

    x = 0
    while x < 10000000:
        x += 1


def experiment(target, n_workers):
    """
    run the target function serially, using threads,
    using process and output the run time
    """
    # Run tasks serially
    start_time = time.time()
    for _ in range(n_workers):
        target()
    
    end_time = time.time()
    print("Serial time=", end_time - start_time)
    print()

    # Run tasks using processes
    start_time = time.time()
    processes = [multiprocessing.Process(target = target) for _ in range(n_workers)]
    for process in processes:
        process.start()

    for process in processes:
        process.join()

    end_time = time.time()
    print("Parallel time=", end_time - start_time)
    print()

    # Run tasks using threads
    start_time = time.time()
    threads = [threading.Thread(target = target) for _ in range(n_workers)]
    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()

    end_time = time.time()
    print("Threads time=", end_time - start_time)

In [13]:
n_workers = 8
experiment(target = only_sleep, n_workers = n_workers)

Process Name: MainProcess, Thread Name: MainThread
Process Name: MainProcess, Thread Name: MainThread
Process Name: MainProcess, Thread Name: MainThread
Process Name: MainProcess, Thread Name: MainThread
Process Name: MainProcess, Thread Name: MainThread
Process Name: MainProcess, Thread Name: MainThread
Process Name: MainProcess, Thread Name: MainThread
Process Name: MainProcess, Thread Name: MainThread
Serial time= 32.00231456756592

Process Name: Process-13, Thread Name: MainThread
Process Name: Process-14, Thread Name: MainThread
Process Name: Process-15, Thread Name: MainThread
Process Name: Process-16, Thread Name: MainThread
Process Name: Process-17, Thread Name: MainThread
Process Name: Process-18, Thread Name: MainThread
Process Name: Process-19, Thread Name: MainThread
Process Name: Process-20, Thread Name: MainThread
Parallel time= 4.079246997833252

Process Name: MainProcess, Thread Name: Thread-9 (only_sleep)
Process Name: MainProcess, Thread Name: Thread-10 (only_sleep)
P

In [14]:
n_workers = 8
experiment(target = crunch_numbers, n_workers = n_workers)

Process Name: MainProcess, Thread Name: MainThread
Process Name: MainProcess, Thread Name: MainThread
Process Name: MainProcess, Thread Name: MainThread
Process Name: MainProcess, Thread Name: MainThread
Process Name: MainProcess, Thread Name: MainThread
Process Name: MainProcess, Thread Name: MainThread
Process Name: MainProcess, Thread Name: MainThread
Process Name: MainProcess, Thread Name: MainThread
Serial time= 2.924499750137329

Process Name: Process-21, Thread Name: MainThread
Process Name: Process-22, Thread Name: MainThreadProcess Name: Process-23, Thread Name: MainThread

Process Name: Process-24, Thread Name: MainThread
Process Name: Process-25, Thread Name: MainThreadProcess Name: Process-26, Thread Name: MainThread
Process Name: Process-28, Thread Name: MainThread
Process Name: Process-27, Thread Name: MainThread

Parallel time= 0.5270814895629883

Process Name: MainProcess, Thread Name: Thread-17 (crunch_numbers)
Process Name: MainProcess, Thread Name: Thread-18 (crunch_